# Desafío 1 - Maratón Behind the Code 2021

### Notebook guia

Este Jupyter Notebook te dará instrucciones para crear una solución introductoria al Desafío 1 de la Maratón. ¡Siéntete libre de editar y mejorar tu solución!

**Ten en cuenta: si estás utilizando Watson Studio, recuerda hacer que tu Notebook sea editable haciendo clic en el botón de edición de arriba.**

![](https://s3.br-sao.cloud-object-storage.appdomain.cloud/maratona-static/edit-notebook.png)

## Exploración de dataset
El primer paso para desarrollar un buen modelo de aprendizaje automático es explorar los datos con los que tenemos que trabajar. Debemos comprender lo mejor posible la relevancia de cada dato para el valor que queremos predecir. Después de todo, la predicción del modelo se basa completamente en los datos con los que se entrenó.

Hay muchas bibliotecas de Python que se pueden utilizar para el procesamiento y la visualización de datos. En este caso usaremos Pandas, Seaborn y Matplotlib.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Primero, cargamos el conjunto de datos del desafío en este notebook. Comencemos con el principal, `LOANS.csv`. Para eso, podemos usar el ícono de activos, disponible en la esquina superior derecha de la pantalla, e insertar el conjunto de datos como un DataFrame Pandas, como en la imagen de abajo.

<img width="300px" src="https://s3.br-sao.cloud-object-storage.appdomain.cloud/maratona-static/load-loans.png" />

Repite este procedimiento para todos los datasets que vas a utilizar.

In [ ]:
# Cargue aqui el dataset

Cambia el nombre de la variable creada con el conjunto de datos para `loans`, para cumplir con los códigos a continuación.

In [ ]:
loans = df_cargado

Podemos usar los métodos .info () y .describe () para obtener información básica sobre la cantidad actual de datos, sus tipos y valores.

In [ ]:
loans.info()

In [ ]:
loans.describe()

La variable de destino para este desafío es `ALLOW`, es decir, si se debe permitir o no un préstamo, según la información proporcionada. Echemos un vistazo a cómo se distribuye esta variable.

In [ ]:
risk_plot = sns.countplot(data=loans, x='ALLOW', order=loans['ALLOW'].value_counts().index)
plt.show()

Siéntete libre de ver la distribución de otras columnas en el conjunto de datos, usar los otros conjuntos de datos, explorar correlaciones entre variables y más.

In [ ]:
# Adicione suas explorações

## Procesamiento de datos

Una vez que hemos explorado los datos, comprendemos la importancia de cada columna y podemos hacer cambios en ellas para obtener un mejor resultado. Aquí, vamos a hacer un procesamiento simple, para eliminar del conjunto de datos las líneas a las que les falta algún valor. Esta técnica no es necesariamente la mejor para usar en el desafío, es solo un ejemplo de cómo manejar el conjunto de datos.

Para procesamientos más avanzados, como modificar columnas o crear columnas nuevas, ve a continuación en Notebook, donde explicamos cómo usar `Pipelines`, desde la biblioteca `sklearn`, para realizar transformaciones de datos.

In [ ]:
clean_df = loans.dropna()
clean_df.count()

Podemos observar que ahora temos un dataset "limpio", pero perdimos algunos datos al eliminar filas donde faltaba al menos una columna.

Al observar la ejecución del método `.info()` anterior, podemos ver que hay tres columnas de tipo `object`. El modelo `scikit-learn` que vamos a utilizar no es capaz de procesar dicha variable. Entonces, para continuar con el experimento, eliminemos esta columna. Recomendamos que utilice alguna técnica para manejar variables categóricas, como _one-hot encoding_, en lugar de eliminar la columna.

También eliminaremos la columna `ID`, ya que sabemos que no es información útil para la predicción (es solo un número que identifica a un cliente).

In [ ]:
object_columns = ['INSTALLMENT_PLANS', 'LOAN_PURPOSE', 'OTHERS_ON_LOAN']
clean_df = clean_df.drop(object_columns, axis=1)
clean_df = clean_df.drop('ID', axis=1)

In [ ]:
clean_df.info()

## Creación del modelo

Con los datos listos, podemos seleccionar un modelo de Machine Learning para entrenar con nuestros datos. En este ejemplo, vamos a utilizar un modelo de clasificación básico, el árbol de decisiones.

Para poder evaluar el rendimiento de nuestro modelo, dividamos los datos que tenemos entre los datos de entrenamiento y de prueba, y luego, después del entrenamiento, veremos cómo le va con las predicciones.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

A continuación, separamos los datos que queremos predecir de los datos que usamos como información para la predicción.

In [ ]:
features = ['PAYMENT_TERM', 'INSTALLMENT_PERCENT', 'LOAN_AMOUNT']
target = ['ALLOW']

X = clean_df[features]
y = clean_df[target]

In [ ]:
test_pct = 0.3 # Separaremos 30% de los dados para testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_pct)

model = DecisionTreeClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Exactitud del modelo (número de predicciones asertadas sobre el número total de pruebas): {acc}")

Aunque estamos usando solo unas pocas variables del conjunto de datos cargado, el desafío espera un modelo que acepte todas las variables de los conjuntos de datos disponibles. Entonces, usemos un transformador para transformar los datos de entrada, eliminando las columnas que no queremos, antes de enviarlo a nuestro modelo. Por lo tanto, vamos a crear un `Pipeline`, que usa el transformador como entrada, y nuestro modelo a continuación.

Depende de ti unir los otros conjuntos de datos disponibles y usarlos también para las predicciones en el modelo, en lugar de eliminar las columnas.

## Sobre Pipelines

Un `Pipeline`, de la librería `scikit-learn`, consta de una serie de pasos donde realizamos transformaciones de datos. Las transformaciones están definidas por clases que siempre deben tener **dos métodos**:

- **fit**: Un método que recibe datos de entrenamiento y devuelve la propia instancia de clase. Se aplica cuando se entrena para usar una canalización para entrenar un modelo.
- **transform**: Un método que toma un conjunto de datos como entrada y debe devolver otro conjunto de datos, transformado. Se aplica a cada paso del Pipeline, recibiendo los datos del paso anterior y transformándolos.

Vea a continuación una representación gráfica de cómo funciona un Pipeline:

![](https://s3.br-sao.cloud-object-storage.appdomain.cloud/maratona-static/pipeline-es.png)

En este Notebook, creamos un Pipeline muy similar al ejemplo anterior, con dos etapas:

- **drop_columns**: Remueve las columnas indeseadas del conjunto de datos de entrada.
- **classification**: Alimenta un modelo de clasificación con los datos obtenidos de la etapa de **drop_columns**, pudiendo ser tanto para entrenamiento como para obtener una predicción. 

## Creación de Pipelines con scikit-learn

Para crear un modelo capaz de hacer transformaciones con los datos de entrada, vamos a crear un `Pipeline` de `scikit-learn` y aplicar nuestras transformaciones dentro de sus etapas.

A continuación, definimos un transformador de ejemplo, que eliminará las columnas pasadas como parámetro en su inicialización:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline


# Un transformador para remover columnas indeseadas
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Primero realizamos la cópia del DataFrame 'X' de entrada
        data = X.copy()
        # Retornamos um nuevo dataframe sin las colunmas indeseadas
        return data.drop(labels=self.columns, axis='columns')

Tanto el método `fit` como `transform` deben definirse, incluso si no van a hacer nada diferente, como en el caso de ese `fit`.

Asimismo, puedes crear otros transformadores, para otros fines, siempre heredando de las clases `BaseEstimator` y `TransformerMixin`. Puedes utilizar un transformador para, por ejemplo, crear nuevas columnas, editar tipos de datos de columnas existentes, etc.

Ahora, vamos a crear un Pipeline para utilizar nuestro modelo, conservando todas las columnas esperadas para el desafío y removiendo las que no queremos usar.

In [ ]:
challenge_columns = ['ID', 'CHECKING_BALANCE', 'PAYMENT_TERM', 'CREDIT_HISTORY',
       'LOAN_PURPOSE', 'LOAN_AMOUNT', 'EXISTING_SAVINGS',
       'EMPLOYMENT_DURATION', 'INSTALLMENT_PERCENT', 'SEX', 'OTHERS_ON_LOAN',
       'CURRENT_RESIDENCE_DURATION', 'PROPERTY', 'AGE', 'INSTALLMENT_PLANS',
       'HOUSING', 'EXISTING_CREDITS_COUNT', 'JOB_TYPE', 'DEPENDENTS',
       'TELEPHONE', 'FOREIGN_WORKER', 'ALLOW']

unwanted_columns = list((set(challenge_columns) - set(target)) - set(features)) # Remover todas las colunmas que no son features do nuestro modelo

In [ ]:
# Creando una instancia del transformador, pasando como parámetro las colunmas que no queremos
drop_columns = DropColumns(unwanted_columns)


# Creando un Pipeline, adicionando nuestro transformador seguido de un modelo de árbol de decisión
skl_pipeline = Pipeline(steps=[('drop_columns', drop_columns), ('classification', model)])

¡Listo! Este Pipeline ahora está listo para recibir todas las variables de desafío, aunque el modelo solo use algunas.

## Deploy del modelo para Watson Machine Learning (WML)

Ahora que tenemos el modelo listo para su publicación, queremos ponerlo en línea para que el sistema de la Maratón pueda probarlo :)

Para ello, utilizaremos la biblioteca `IBM Watson Machine Learning`, que le permite encapsular modelos de Machine Learning en APIs REST.

In [ ]:
# Instalar la biblioteca WML
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

Si aún no lo ha creado, cree un servicio de Machine Learning aquí https://cloud.ibm.com/catalog/services/machine-learning.

Ingresa tus credenciales para el servicio en la celda a continuación.

En `location`, ingresa el ID de la región donde se encuentra su servicio de WML instanciado, de acuerdo con las posibilidades de abajo:

-	Dallas - `us-south`
-	London - `eu-gb`
-	Frankfurt - `eu-de`
-	Tokyo - `jp-tok`

Para la API key, generela aqui: https://cloud.ibm.com/iam/apikeys. No la compartas con nadie! Una API key da acesso a su cuenta de IBM Cloud.

In [ ]:
api_key = 'INSIRA SUA API KEY AQUI'
location = 'us-south' # En caso de WML estar en una región diferente, altere esta linea

wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

client = APIClient(wml_credentials)

Crea un espacio para guardar tu modelo. Puedes crearlo aqui: https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas

Cuando crees tu espacio, **asocia la instancia de tu servicio de WML al espacio!** Sin asociar, no conseguiras efectuar el deploy.

In [ ]:
# Lista espacios creados en su instancia de WML
client.spaces.list(limit=10)

Copia el ID de tu espacio creado para el desafío y pégualo a continuación para usarlo. Deberías ver el mensaje 'SUCCESS' si el espacio está configurado correctamente.

In [ ]:
space_id = 'cole aqui'
client.set.default_space(space_id)

## Utilización de Pipeline dentro de Watson Machine Learning (WML)

Para utilizar un Pipeline en WML con transformadores customizados, son necesarios algunos pasos adicionales:

1.	Crea un paquete en Python que contenga el transformador personalizado.
2.	Carga ese paquete con el transformador en un repositório en WML;
3.	Crea una especificación de software, con este paquete personalizado, que se utilizará como tiempo de ejecución del modelo en WML.

Como exemplo, vamos utilizar un paquete ya listo, disponible aqui: https://github.com/vnderlev/watson-sklearn-transforms. Para configurar un paquete de Python, son necesarios algunos otros archivos, pero la lógica del transformador creado se encuentra en [este archivo](https://github.com/vnderlev/watson-sklearn-transforms/blob/master/my_custom_sklearn_transforms/sklearn_transformers.py). En este caso, este es el mismo transformador que definimos aquí, excluirá del conjunto de datos las columnas pasadas como parámetros en su inicialización.

Abajo, vamos a bajar este paquete de GitHub e instalarlo en Python.

In [ ]:
!rm -rf watson-sklearn-transforms # Remover carpeta en caso de que ya exista
!git clone https://github.com/vnderlev/watson-sklearn-transforms # Clonar el repositório con el pacote
!zip -r drop-columns.zip watson-sklearn-transforms # Zippear el paquete
!pip install drop-columns.zip # Instalar el paquete zippeado

Vamos ahora a recrear nuestro Pipeline utilizando este paquete instalado.

In [ ]:
from my_custom_sklearn_transforms.sklearn_transformers import DropColumns

drop_columns = DropColumns(unwanted_columns)

pipeline = Pipeline(steps=[('drop_columns', drop_columns), ('classification', model)])

Vamos ahora a subir el transformador customizado que bajamos para WML.

In [ ]:
# Metadatos para el paquete customizado
meta_prop_pkg_extn = {
    client.package_extensions.ConfigurationMetaNames.NAME: "Drop_Columns",
    client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "Extensión para remover columnas",
    client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip"
}

# Subir el paquete
pkg_extn_details = client.package_extensions.store(meta_props=meta_prop_pkg_extn, file_path="drop-columns.zip")

# Guardar las informaciones sobre el paquete
pkg_extn_uid = client.package_extensions.get_uid(pkg_extn_details)
pkg_extn_url = client.package_extensions.get_href(pkg_extn_details)

Ahora creemos una especificación de software con nuestro paquete personalizado para que lo use WML. Si estás utilizando un software que no sea `Python 3.8` o una biblioteca que no sea `scikit-learn`, puedes consultar la lista de especificaciones de software compatibles con WML: https://dataplatform.cloud.ibm.com/docs/content/wsj/wmls/wmls-deploy-python-types.html?context=analytics&audience=wdp

In [ ]:
base_sw_spec_uid = client.software_specifications.get_uid_by_name("default_py3.8")

# Si desea utilizar un software que no sea Python 3.8 como base, eche un vistazo a los disponibles con la línea a continuación
# client.software_specifications.list(limit=100)

In [ ]:
# Metadatos de la nueva especificación de software
meta_prop_sw_spec = {
    client.software_specifications.ConfigurationMetaNames.NAME: "sw_spec_drop_columns",
    client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "Software specification for DropColumns",
    client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: {"guid": base_sw_spec_uid}
}

# Creando la nueva especificacion de software y obteniendo su ID
sw_spec_details = client.software_specifications.store(meta_props=meta_prop_sw_spec)
sw_spec_uid = client.software_specifications.get_uid(sw_spec_details)

# Agregar el paquete personalizado a la nueva especificación
client.software_specifications.add_package_extension(sw_spec_uid, pkg_extn_uid)

Finalmente, vamos a publicar el pipeline utilizando la especificación de software customizada que creamos.

In [ ]:
# Metadatos del modelo
model_props = {
    client.repository.ModelMetaNames.NAME: "Modelo com Pipeline customizada",
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sw_spec_uid
}

# Publicando el Pipeline como um modelo
published_model = client.repository.store_model(model=pipeline, meta_props=model_props)
published_model_uid = client.repository.get_model_uid(published_model)
client.repository.get_details(published_model_uid)

Su modelo ahora está guardado. Vamos ahora a dejarlo disponible online, para que podamos testearlo:

In [ ]:
# Metadatos para publicación del modelo
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Publicación de modelo customizado",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

# Publicar
created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

## Felicitaciones!

Su modelo está ahora publicado. Cuando este listo para enviar el desafio, puedes acceder a https://maratona.dev/challenge/1, y utilizar las credenciales abajo para realizar la entrega. Recuerda revisar todas las instrucciones en el [README](https://github.com/maratonadev/desafio-1-2021) antes de entregar!

In [ ]:
deployment_uid = client.deployments.get_uid(created_deployment)

print(f"Credenciales para el envio (no compartir estos datos con nadie!)\n\nAPI key: {api_key}\nDeployment ID: {deployment_uid}")